In [3]:
import time
import re
from typing import Any
from specparser.amt import loader, schedules, table, tickers, strings


amt_path = "../data/amt.yml"
prices_path = "../data/prices.parquet"
chain_path = "../data/futs.csv"
override_path = "../data/overrides.csv"

_MEMOIZE_ENABLED = True

_ASSET_STRADDLE_TICKER_CACHE = {}
_ASSET_STRADDLE_TICKER_COLUMNS=["name","ticker","field"]

def split_ticker(ticker,year,month):
        if ":" in ticker:
            ticker_pre,ym,ticker_post =  ticker.split(":",2)
            if ym<f"{year}-{month:02d}":
                return ticker_pre
            else:
                return ticker_post
        else:
            return ticker

def make_fut_ticker(fut_code,fut_month_map,min_year_offset,market_code,qualifier,year,month):
    month_map = "FGHJKMNQUVXZ"
    fut_month_code = fut_month_map[month - 1]
    opt_month_code = month_map[month - 1]
    year_offset = max(1 if fut_month_code < opt_month_code else 0, min_year_offset)
    fut_year = year + year_offset
    return  f"{fut_code}{qualifier}{fut_month_code}{fut_year} {market_code}"

def asset_straddle_ticker_key(
    asset: str, 
    strym: str, 
    ntrc:str,
    vol:dict,
    hedge:dict
):
    if vol["Source"]=="BBG_LMEVOL":
        cache_key = asset+"|"+strym+"|"+ntrc
    elif hedge["Source"]=="fut":
        cache_key = asset+"|"+strym+"|"+ntrc
    elif hedge["Source"]=="nonfut" and ":" in hedge["Ticker"]:
        cache_key = asset+"|"+strym+"|"+ntrc
    elif hedge["Source"]=="cds" and ":" in hedge["hedge"]:
        cache_key = asset+"|"+strym+"|"+ntrc
    elif hedge["Source"]=="cds" and ":" in hedge["hedge1"]:
        cache_key = asset+"|"+strym+"|"+ntrc
    else: 
        cache_key = asset
    return cache_key

def get_asset_straddle_tickers(asset: str, strym: str, ntrc:str, amt_path: str) -> dict[str,Any]:

    ticker_dict = dict(
        orientation="row",
        columns=_ASSET_STRADDLE_TICKER_COLUMNS,
        rows = []
    )

    asset_data = loader.get_asset(amt_path,asset)
    if asset_data is None : return ticker_dict
    hedge = asset_data["Hedge"]
    if hedge is None: return ticker_dict
    vol = asset_data["Vol"]
    if vol is None: return ticker_dict

    if ntrc=="F":
        if vol.get("Far","")=="": return ticker_dict
        if vol.get("Near","")==vol.get("Far",""): return ticker_dict
        if vol.get("Far","")=="NONE": return ticker_dict

    cache_key = asset_straddle_ticker_key(asset,strym,ntrc,vol,hedge)

    if _MEMOIZE_ENABLED and cache_key in _ASSET_STRADDLE_TICKER_CACHE:
        return _ASSET_STRADDLE_TICKER_CACHE[cache_key]

    year, month = int(strym[0:4]), int(strym[5:7])

    if vol["Source"]=="BBG" and ntrc=="N":
       ticker_dict["rows"].append(["vol",vol["Ticker"],vol["Near"]])

    if vol["Source"]=="BBG" and ntrc=="F":
        ticker_dict["rows"].append(["vol",vol["Ticker"],vol["Far"]])

    if vol["Source"]=="CV":
        ticker_dict["rows"].append(["vol",vol["Near"],"none"])

    if vol["Source"]=="BBG_LMEVOL":
        vol_ticker =  make_fut_ticker(
            fut_code=hedge["fut_code"],
            fut_month_map=hedge["fut_month_map"],
            min_year_offset=hedge["min_year_offset"],
            market_code=hedge["market_code"],
            qualifier="R",
            year=year,
            month=month,
        )
        ticker_dict["rows"].append(["vol",vol_ticker,"PX_LAST"])

    if hedge["Source"]=="fut":
        fut_ticker = make_fut_ticker(
            fut_code=hedge["fut_code"],
            fut_month_map=hedge["fut_month_map"],
            min_year_offset=hedge["min_year_offset"],
            market_code=hedge["market_code"],
            qualifier="",
            year=year,
            month=month,
        )
        ticker_dict["rows"].append(["hedge",fut_ticker,"PX_LAST"])

    if hedge["Source"]=="nonfut":
        ticker_dict["rows"].append(["hedge",split_ticker(hedge["Ticker"],year,month),hedge["Field"]])

    if hedge["Source"]=="cds":
        ticker_dict["rows"].append(["hedge",split_ticker(hedge["hedge"],year,month),"PX_LAST"])
        ticker_dict["rows"].append(["hedge1",split_ticker(hedge["hedge1"],year,month),"PX_LAST"])

    if hedge["Source"]=="calc":
        ticker_dict["rows"].append(["hedge1",f"{hedge['ccy']}_fsw0m_{hedge['tenor']}",""])
        ticker_dict["rows"].append(["hedge2",f"{hedge['ccy']}_fsw6m_{hedge['tenor']}",""])
        ticker_dict["rows"].append(["hedge3",f"{hedge['ccy']}_pva0m_{hedge['tenor']}",""])
        ticker_dict["rows"].append(["hedge4",f"{hedge['ccy']}_pva6m_{hedge['tenor']}",""])
    
    if _MEMOIZE_ENABLED:
        _ASSET_STRADDLE_TICKER_CACHE[cache_key] = ticker_dict

    return ticker_dict

def _ym_range(ntry: str, xpry: str) -> list[str]:
    sy, sm = int(ntry[:4]), int(ntry[5:7])
    ey, em = int(xpry[:4]), int(xpry[5:7])
    start = sy * 12 + (sm - 1)
    end   = ey * 12 + (em - 1)
    # months: start .. end inclusive  => range(start, end+1)
    return [f"{m//12:04d}-{(m%12)+1:02d}" for m in range(start, end + 1)]

def find_assets_straddles_tickers(pattern: str, ntry: str, xpry:str, amt_path: str):

    assets = [asset for _, asset in loader._iter_assets(amt_path, live_only=True, pattern=pattern)]
    months = _ym_range(ntry, xpry)
    ntrcs  = ("N", "F")

    # find column indices once
    cols = _ASSET_STRADDLE_TICKER_COLUMNS
    i_ticker = cols.index("ticker")
    i_field  = cols.index("field")

    rows_out: list[list[str]] = []
    seen: set[tuple[str, str]] = set()

    for asset in assets:
        for ym in months:
            for ntrc in ntrcs:
                straddles = get_asset_straddle_tickers(asset, ym, ntrc, amt_path)
                for row in straddles["rows"]:
                    key = (row[i_ticker],row[i_field])
                    if key in seen: 
                        #print(f"dupe: {row[i_ticker]},{row[i_field]}")
                        continue
                    seen.add(key)
                    rows_out.append(row)

    return {
        "columns": _ASSET_STRADDLE_TICKER_COLUMNS,
        "orientation": "row",
        "rows": rows_out,
    }

            



In [11]:
#asset_tickers = get_asset_straddle_tickers("LA Comdty TKRZ","2025-01","N",amt_path)
asset_tickers = table.table_bind_rows(
    get_asset_straddle_tickers("LA Comdty","2025-01","N",amt_path),
    get_asset_straddle_tickers("LA Comdty","2025-01","F",amt_path),
    get_asset_straddle_tickers("XB Comdty","2001-01","N",amt_path),
    get_asset_straddle_tickers("SI Comdty","2001-01","F",amt_path),
    get_asset_straddle_tickers("SB Comdty","2001-01","N",amt_path),
    get_asset_straddle_tickers("S Comdty","2001-01","F",amt_path),
    get_asset_straddle_tickers("IBOXUMAE Curncy","2025-01","N",amt_path),
    get_asset_straddle_tickers("IBOXUMAE Curncy","2025-02","N",amt_path),
    get_asset_straddle_tickers("IBOXUMAE Curncy","2025-03","N",amt_path),
    get_asset_straddle_tickers("EURUSD Curncy","2025-01","N",amt_path),
    get_asset_straddle_tickers("USSWAP10 Curncy","2025-01","N",amt_path)
)
print("-"*40)
print(_ASSET_STRADDLE_TICKER_CACHE.keys())
table.show_table(asset_tickers)

----------------------------------------
dict_keys(['LA Comdty|2025-01|N', 'LA Comdty|2025-01|F', 'XB Comdty|2001-01|N', 'SI Comdty|2001-01|F', 'SB Comdty|2001-01|N', 'S Comdty|2001-01|F', 'IBOXUMAE Curncy|2025-01|N', 'IBOXUMAE Curncy|2025-02|N', 'IBOXUMAE Curncy|2025-03|N', 'EURUSD Curncy', 'USSWAP10 Curncy'])


,name,ticker,field
0,vol,LA1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
1,hedge,LAF2025 Comdty,PX_LAST
2,vol,LA1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
3,hedge,LAF2025 Comdty,PX_LAST
4,vol,XB1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
5,hedge,XBG2001 Comdty,PX_LAST
6,vol,SI1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
7,hedge,SIH2001 Comdty,PX_LAST
8,vol,SB1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
9,hedge,SBH2001 Comdty,PX_LAST


In [8]:
asset_tickers=find_assets_straddles_tickers(".","2001-01","2026-01",amt_path)
print(len(_ASSET_STRADDLE_TICKER_CACHE.keys()))
table.show_table(asset_tickers)

12507


,name,ticker,field
0,vol,LA1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
1,hedge,LAF2001 Comdty,PX_LAST
2,vol,LA1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
3,hedge,LAG2001 Comdty,PX_LAST
4,hedge,LAH2001 Comdty,PX_LAST
...,...,...,...
4557,hedge,CSCO US Equity,PX_LAST
4558,vol,DVN US Equity,30DAY_IMPVOL_100.0%MNY_DF
4559,hedge,DVN US Equity,PX_LAST
4560,vol,LOW US Equity,30DAY_IMPVOL_100.0%MNY_DF


In [53]:
from itertools import product
tst=[ "x" ] * 10
print(len(tst))
for _ , _ ,_, _, _ in product(range(25),range(12),range(30),range(200), range(4)):
    pass
    tst[0]="y"
print(len(tst))



10
10
